In [1]:
%cd C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor

import sys
from pathlib import Path
from datetime import date

from src.pipeline.create_training_dataset import *

import xarray as xr
from tqdm import tqdm

import yaml
from pathlib import Path
import logging

from src.utils.grid_utils import Grid
import src.data_src.point_data_sources as pds
import src.data_src.gridded_data_sources as gds
from src.utils.data_utils import DataMapping

C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor


In [2]:
dm_cetb = DataMapping(r'data\raw\CETB_AMSR2', dataset='CETB')

In [2]:
dm_ERA5 = DataMapping(r'data\raw\ERA5_all-param_daily_perMonth_1hourly-sample', dataset='ERA5')

In [4]:
dm_ERA5.get_by_channel('t2m')

,date,channel,filename
3,2013-02-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
22,2013-03-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
41,2013-04-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
60,2017-03-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
79,2017-04-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
98,2019-03-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
117,2019-04-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
136,2020-10-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
155,2020-11-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...
174,2020-12-01,t2m,data\raw\ERA5_all-param_daily_perMonth_1hourly...


In [30]:
#project_root = Path(__file__).resolve().parents[2]
config_path = "configs/pipeline_configs/create_training_dataset.yaml"
config = yaml.safe_load(open(config_path, "r"))

target_grid = Grid.from_predefined(config['grid'])

In [18]:
# %% make point data dataframe
# Load point data sources

point_sources = []
for source_cfg in tqdm(config['point_data_sources'], desc="Loading point data sources"):
    for sid_cfg in config['point_data_sources'][source_cfg]['secondary_id']:
        source = load_point_data_source(
            filepath=config['point_data_sources'][source_cfg]['secondary_id'][sid_cfg]['filepath'],
            secondary_id=config['point_data_sources'][source_cfg]['secondary_id'][sid_cfg]['id']
        )
        point_sources.append(source)

# Grid and merge point data sources
gridded_point_data = grid_and_merge_point_data_sources(point_sources, target_grid)

Processing dates: 100%|██████████| 4/4 [00:06<00:00,  1.74s/date]


In [37]:
# %% match to gridded data sources
dates = gridded_point_data.data['time'].dt.date.unique()

ERA5_mapping = gds.map_ERA5_files(config['ERA5']['directory'], channels=config['ERA5']['channels'])
CETB_mapping = gds.map_CETB_files(config['CETB']['directory'], channels=config['CETB']['channels'])


In [39]:
match

<xarray.Dataset> Size: 17MB
Dimensions:  (time: 1, points: 124123)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-11-02
    x        (points) float64 993kB -6.266e+05 -6.234e+05 ... -1.28e+06
    y        (points) float64 993kB 2.234e+05 2.234e+05 ... 1.523e+06 1.523e+06
Dimensions without coordinates: points
Data variables: (12/31)
    u10      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    v10      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    d2m      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    t2m      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    e        (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    fal      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    ...       ...
    89H      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    89V      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    18H      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    18V      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    23H      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>
    23V      (time, points) float32 496kB dask.array<chunksize=(1, 124123), meta=np.ndarray>

In [50]:
ERA5_dict = yaml.safe_load(open('configs/ERA5_variable_dictionary.yaml'))


In [64]:
import pandas as pd
ERA5_dict_df = pd.DataFrame(ERA5_dict).T
ERA5_dict_df.index.name = 'short_name'
ERA5_dict_df=ERA5_dict_df.reset_index()
ERA5_dict_df = ERA5_dict_df.set_index('filename')
ERA5_dict_df

,short_name,long_name
filename,,
10m_u_component_of_wind_stream-oper_daily-mean.nc,10u,10m_u_component_of_wind
10m_v_component_of_wind_0_daily-mean.nc,10v,10m_v_component_of_wind
2m_dewpoint_temperature_0_daily-mean.nc,d2m,2m_dewpoint_temperature
evaporation_0_daily-mean.nc,e,evaporation
forecast_albedo_0_daily-mean.nc,fal,forecast_albedo
ice_temperature_layer_1_0_daily-mean.nc,ictl1,ice_temperature_layer_1
ice_temperature_layer_2_0_daily-mean.nc,ictl2,ice_temperature_layer_2
ice_temperature_layer_3_0_daily-mean.nc,ictl3,ice_temperature_layer_3
ice_temperature_layer_4_0_daily-mean.nc,ictl4,ice_temperature_layer_4


In [84]:
b = ERA5_dict_df.index.tolist()

In [88]:
b = '10m_v_component_of_wind_0_daily-mean.nc'

In [91]:
ERA5_dict_df.loc[b]['short_name']

'10v'

In [82]:
ERA5_dict_df.loc[['10m_v_component_of_wind_stream-oper_daily-mean.nc']]

KeyError: "None of [Index(['10m_v_component_of_wind_stream-oper_daily-mean.nc'], dtype='object', name='filename')] are in the [index]"

In [74]:
import numpy as np
test = ['10m_u_component_of_wind_stream-oper_daily-mean.nc','10m_v_component_of_wind_stream-oper_daily-mean.nc','2m_dewpoint_temperature_stream-oper_daily-mean.nc']

ERA5_dict_df[pd.Series(test)]

KeyError: "None of [Index(['10m_u_component_of_wind_stream-oper_daily-mean.nc',\n       '10m_v_component_of_wind_stream-oper_daily-mean.nc',\n       '2m_dewpoint_temperature_stream-oper_daily-mean.nc'],\n      dtype='object')] are in the [columns]"

In [ ]:
matched_sources = []
for date in tqdm(dates[:2], desc="Matching dates"):
    ERA5_data = gds.load_ERA5_data(ERA5_mapping[date], target_grid)
    CETB_data = gds.load_CETB_data(CETB_mapping[date], target_grid)
    match_data = ERA5_data + CETB_data

    #match = sample_from_gridded_sources(gridded_point_data, match_data)
    gridded_source = match_data
    point_source = gridded_point_data

    c_point_df = point_source.data
    c_point_df = c_point_df[c_point_df['time'].dt.date == date]
    points = xr.Dataset({
        #"time": (("points",), c_point_df['time'].astype('<M8[ns]').values),
        "x":   (("points",), c_point_df['x'].values),
        "y":  (("points",), c_point_df['y'].values),
    })

    match = gridded_source.data.interp(
        #time=points["time"],
        x=points["x"],
        y=points["y"],
        method = 'linear'
    )

    sampled_df = c_point_df.copy()
    for var in match.data_vars:
        sampled_df[var] = match[var].values[0]
    
    #return pds.GriddedPointDataSource(sampled_df, point_source.grid)


    #matched_sources.append(match)

Matching dates:   0%|          | 0/2 [00:16<?, ?it/s]


ValueError: Length of values (1) does not match length of index (141)

In [21]:
full_dataset = pds.GriddedPointDataSource.merge_sources(matched_sources)
full_dataset.to_parquet(config['full_dataset']['directory'] / config['full_dataset']['name'])

NameError: name 'matched_sources' is not defined

In [ ]:
# %% dataset splits
train_set, val_set, test_set = split_dataset(
    full_dataset,
    val_frac=config['split_params']['val_fraction'],
    val_equalization=config['split_params']['val_equalization'],
    test_period=config['split_params']['test_period']
)

train_set.to_parquet(Path(config['train_set']['directory']) / config['train_set']['name'])
if val_set is not None:
    val_set.to_parquet(Path(config['val_set']['directory']) / config['val_set']['name'])
if test_set is not None:
    test_set.to_parquet(Path(config['test_set']['directory']) / config['test_set']['name'])

In [ ]:
import yaml

import src.data_src.point_data_sources as pds
import src.data_src.gridded_data_sources as gds
import src.utils.data_utils as data_utils
from src.utils.grid_utils import Grid


# %% Point data sources
def load_point_data_source(filepath, secondary_id):
    if secondary_id in ['2020-2021', '2021-2022']:
        return pds.C2I.C2I(filepath, secondary_id=secondary_id)
    elif secondary_id == 'IDCSI4':
        return pds.OIB.OIB(filepath)
    elif secondary_id == 'ICEBIRD':
        return pds.AEM_AWI.AEM_AWI_ICEBIRD(filepath)
    elif secondary_id == 'PARARCMIP':
        return pds.AEM_AWI.AEM_AWI_PARARCMIP(filepath)
    else:
        raise ValueError(f"Unsupported secondary_id: {secondary_id}")


def grid_and_merge_point_data_sources(sources, target_grid_id):
    gridded_sources = []
    for source in sources:
        gridded_sources.append(source.resample_bucket(target_grid_id))
    
    # Merge all gridded sources into one
    return pds.base.GriddedPointDataSource.merge_sources(gridded_sources)


# %% Gridded data sources
def load_CETB_channels(CETB_mapping, date, grid, channels=None):
    files = CETB_mapping[date]
    if channels is not None:
        channel_files = [f for f in files if Path(f).stem.split('_')[7] in channels]
    else:
        channel_files = files

    cetb_scenes = []
    for file in channel_files:
        cetb_scene = gds.CETBScene.from_files([file])
        cetb_scene.regrid(grid)
        cetb_scenes.append(cetb_scene)
    
    cetb_ds = gds.GriddedDataSource.merge(cetb_scenes)
    return cetb_ds

def load_ERA5_data(ERA5_mapping, date, grid, variables = None):
    ERA5_dict = yaml.safe_load(open(project_root / 'configs/ERA5_variable_dictionary.yaml'))
    files = ERA5_mapping[date]
    if variables is not None:
        variable_files = [f for f in files if ERA5_dict[var]['filename'] in f for var in variables]
    else:
        variable_files = files

    era5_ds = gds.ERA5Scene.from_files(variable_files)
    era5_ds.regrid(grid)
    return era5_ds

In [ ]:
CETB_mapping = data_utils.map_CETB_file_dates(r"C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor\data\raw\CETB_AMSR2")
ERA5_mapping = data_utils.map_ERA5_file_dates(r"C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor\data\raw\ERA5_all-param_daily_perMonth_1hourly-sample")

In [ ]:
grid = Grid.from_predefined('EASE2_N3.125km')
era5_ds = load_ERA5_data(ERA5_mapping, date(2020, 11, 15), grid=grid, variables=None)
cetb_ds = load_CETB_channels(CETB_mapping, date(2020, 11, 15), grid=grid, channels=None)

ds = era5_ds + cetb_ds
ds

<xarray.Dataset> Size: 4GB
Dimensions:  (time: 1, x: 5760, y: 5760)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-11-15
  * x        (x) float64 46kB -8.998e+06 -8.995e+06 ... 8.995e+06 8.998e+06
  * y        (y) float64 46kB 8.998e+06 8.995e+06 ... -8.995e+06 -8.998e+06
Data variables: (12/32)
    u10      (time, y, x) float32 133MB dask.array<chunksize=(1, 5760, 5760), meta=np.ndarray>
    v10      (time, y, x) float32 133MB dask.array<chunksize=(1, 5760, 5760), meta=np.ndarray>
    d2m      (time, y, x) float32 133MB dask.array<chunksize=(1, 5760, 5760), meta=np.ndarray>
    t2m      (time, y, x) float32 133MB dask.array<chunksize=(1, 5760, 5760), meta=np.ndarray>
    e        (time, y, x) float32 133MB dask.array<chunksize=(1, 5760, 5760), meta=np.ndarray>
    fal      (time, y, x) float32 133MB dask.array<chunksize=(1, 5760, 5760), meta=np.ndarray>
    ...       ...
    89H      (time, y, x) float32 133MB dask.array<chunksize=(1, 1440, 1440), meta=np.ndarray>
    89V      (time, y, x) float32 133MB dask.array<chunksize=(1, 1440, 1440), meta=np.ndarray>
    18H      (time, y, x) float32 133MB dask.array<chunksize=(1, 1440, 5760), meta=np.ndarray>
    18V      (time, y, x) float32 133MB dask.array<chunksize=(1, 1440, 5760), meta=np.ndarray>
    23H      (time, y, x) float32 133MB dask.array<chunksize=(1, 1440, 5760), meta=np.ndarray>
    23V      (time, y, x) float32 133MB dask.array<chunksize=(1, 1440, 5760), meta=np.ndarray>